In [1]:
import json
import os

class Player:
    """Base class for player data (loads & saves from JSON)."""
    def __init__(self, name, role):
        self.name = name
        self.role = role
        self.health = 100
        self.mana = 50
        self.level = 1
        self.stats = {"strength": 5, "agility": 3, "intelligence": 4}
        self.inventory = []

    def save_profile(self, filename="data/player.json"):
        """Saves player data to JSON, ensuring the folder exists."""
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "w") as f:
            json.dump(self.__dict__, f, indent=4)
        print("💾 Player data saved!")

    def load_profile(self, filename="data/player.json"):
        """Loads player data from JSON."""
        try:
            with open(filename, "r") as f:
                data = json.load(f)
                self.__dict__.update(data)
            print(f"📂 Loaded player: {self.name}, Level {self.level}, Role: {self.role}")
        except FileNotFoundError:
            print("⚠ No saved player profile found. Starting fresh.")

# 🎭 Player Subclasses

class Warrior(Player):
    """Warrior class - Strong melee fighter with high health."""
    def __init__(self, name):
        super().__init__(name, role="Warrior")
        self.health = 120  # More health
        self.stats["strength"] += 5  # Extra melee power
        self.stats["agility"] += 1  # Lower agility

class Mage(Player):
    """Mage class - Uses magic instead of brute force."""
    def __init__(self, name):
        super().__init__(name, role="Mage")
        self.mana = 100  # More mana
        self.stats["intelligence"] += 6  # High magic power
        self.stats["strength"] -= 2  # Weak physical attacks

class Rogue(Player):
    """Rogue class - Fast, agile fighter with critical hits."""
    def __init__(self, name):
        super().__init__(name, role="Rogue")
        self.stats["agility"] += 7  # Fast movement & dodging
        self.stats["strength"] += 2  # Decent attack power
        self.stats["intelligence"] -= 2  # Lower magic ability

# Example Usage
if __name__ == "__main__":
    player = Mage("Eldrin")
    player.save_profile()

💾 Player data saved!


In [5]:
class Inventory:
    """Manages the player's inventory."""
    def __init__(self):
        self.items = []

    def add_item(self, item_name, item_type, **attributes):
        """Add an item to the inventory."""
        item = {"name": item_name, "type": item_type, **attributes}
        self.items.append(item)
        print(f"👜 {item_name} added to inventory!")

    def use_item(self, player, item_name):
        """Use an item from the inventory."""
        for item in self.items:
            if item["name"].lower() == item_name.lower():
                if item["type"] == "potion":
                    player.health += item.get("heal", 0)
                    print(f"🩹 {player.name} used {item_name} and healed!")
                    self.items.remove(item)
                elif item["type"] == "weapon":
                    print(f"⚔ {player.name} is now wielding {item_name}!")
                return
        print(f"❌ {item_name} not found!")

# Example Usage
inventory = Inventory()
inventory.add_item("Steel Sword", "weapon", damage=20)


👜 Steel Sword added to inventory!


In [6]:

import json
import os

class GameRules:
    """Loads game difficulty settings from JSON."""
    def __init__(self, difficulty="Normal", player_role="Warrior"):
        self.difficulty = difficulty
        self.player_role = player_role
        self.filename = "data/game_rules.json"
        self.load_rules()

    def load_rules(self):
        """Loads difficulty settings from JSON or creates default settings if missing."""
        if not os.path.exists(self.filename):
            self.create_default_rules()  # ✅ Auto-generate game_rules.json if missing

        try:
            with open(self.filename, "r") as f:
                data = json.load(f)
                self.__dict__.update(data.get(self.difficulty, {}))
                self.apply_role_modifiers()
            print(f"🎮 Game Mode: {self.difficulty}, Role: {self.player_role}")
            print(f"⚔ Enemy Damage Multiplier: {self.enemy_damage_multiplier}")
        except FileNotFoundError:
            print("⚠ No game rules file found. Using default settings.")

    def apply_role_modifiers(self):
        """Modifies game rules based on player type."""
        if self.player_role == "Warrior":
            self.enemy_damage_multiplier *= 0.9  # Warriors take less damage
            self.xp_multiplier *= 0.8  # Level up slower
        elif self.player_role == "Mage":
            self.enemy_damage_multiplier *= 1.2  # Mages take more damage
            self.xp_multiplier *= 1.5  # Level up faster
        elif self.player_role == "Rogue":
            self.enemy_damage_multiplier *= 1.0  # Normal damage
            self.item_drop_rate *= 0.8  # Rogues find fewer items

    def create_default_rules(self):
        """Creates default game rules if the file is missing."""
        default_rules = {
            "Easy": {
                "enemy_damage_multiplier": 0.8,
                "xp_multiplier": 1.2,
                "item_drop_rate": 1.5,
                "starting_gold": 200
            },
            "Normal": {
                "enemy_damage_multiplier": 1.0,
                "xp_multiplier": 1.0,
                "item_drop_rate": 1.0,
                "starting_gold": 100
            },
            "Hard": {
                "enemy_damage_multiplier": 1.5,
                "xp_multiplier": 0.8,
                "item_drop_rate": 0.7,
                "starting_gold": 50
            }
        }
        os.makedirs(os.path.dirname(self.filename), exist_ok=True)
        with open(self.filename, "w") as f:
            json.dump(default_rules, f, indent=4)
        print("📜 Default game rules created.")

# Example Usage
rules = GameRules("Hard", "Mage")  # Testing with a Mage
rules.load_rules()


📜 Default game rules created.
🎮 Game Mode: Hard, Role: Mage
⚔ Enemy Damage Multiplier: 1.7999999999999998
🎮 Game Mode: Hard, Role: Mage
⚔ Enemy Damage Multiplier: 1.7999999999999998


In [7]:
import random

def event_generator():
    """Yields random world events."""
    events = ["You found a treasure chest!", "A wild beast appears!", "You discovered an ancient ruin."]
    while True:
        yield random.choice(events)

# Example Usage
exploration = event_generator()
for _ in range(3):
    print(next(exploration))

You found a treasure chest!
A wild beast appears!
You discovered an ancient ruin.


In [9]:
import random
from player import Player

class Combat:
    """Handles turn-based combat mechanics."""
    
    def __init__(self, player):
        self.player = player
        self.enemy = self.generate_enemy()

    def generate_enemy(self):
        """Dynamically generates an enemy with randomized stats."""
        enemies = [
            {"name": "Goblin", "health": 50, "attack": 10, "defense": 2, "evasion": 5},
            {"name": "Orc", "health": 80, "attack": 15, "defense": 4, "evasion": 3},
            {"name": "Dark Mage", "health": 60, "attack": 12, "defense": 3, "evasion": 7}
        ]
        return random.choice(enemies)

    def calculate_damage(self, attacker_strength, enemy_defense):
        """Uses expressions to calculate attack damage dynamically."""
        base_damage = attacker_strength * random.uniform(1.2, 1.8)  # Adds some randomness
        final_damage = max(base_damage - enemy_defense, 1)  # Ensure at least 1 damage
        return round(final_damage, 1)

    def player_attack(self):
        """Player's turn to attack."""
        if random.randint(1, 100) <= self.enemy["evasion"]:  # Enemy dodges attack
            print(f"💨 {self.enemy['name']} dodged the attack!")
            return
        
        damage = self.calculate_damage(self.player.stats["strength"], self.enemy["defense"])
        self.enemy["health"] -= damage
        print(f"🗡 {self.player.name} attacks! {self.enemy['name']} loses {damage} HP.")

        # Special Abilities
        if self.player.role == "Warrior" and random.randint(1, 100) <= 20:
            print(f"💥 {self.player.name} lands a **CRITICAL STRIKE** for double damage!")
            self.enemy["health"] -= damage  # Apply extra hit

        if self.player.role == "Mage" and self.player.mana >= 20:
            print(f"🔥 {self.player.name} casts **Fireball** on {self.enemy['name']}!")
            self.enemy["health"] -= 15
            self.player.mana -= 20  # Spend mana

    def enemy_attack(self):
        """Enemy's turn to attack."""
        if random.randint(1, 100) <= self.player.stats.get("agility", 0) * 2:  # Higher agility = better dodge chance
            print(f"💨 {self.player.name} dodged the enemy attack!")
            return

        damage = self.calculate_damage(self.enemy["attack"], self.player.stats.get("defense", 0))
        self.player.health -= damage
        print(f"⚔ {self.enemy['name']} attacks! {self.player.name} loses {damage} HP.")

    def combat_round(self):
        """Runs a single round of combat until someone is defeated."""
        print(f"\n⚔ A wild {self.enemy['name']} appears!")
        
        while self.player.health > 0 and self.enemy["health"] > 0:
            self.player_attack()
            if self.enemy["health"] <= 0:
                print(f"🏆 {self.enemy['name']} defeated!")
                break

            self.enemy_attack()
            if self.player.health <= 0:
                print(f"💀 {self.player.name} was defeated! Game Over.")
                break


In [ ]:
# 📂 Import Required Modules
from player import Warrior, Mage, Rogue
from inventory import Inventory
from combat import Combat
from world import event_generator
from rules import GameRules

# 📜 Function to Choose Player Type
def choose_player_type():
    """Allows the user to select a player type at game start."""
    print("\n🎭 Choose Your Player Type:")
    print("1. 🛡 Warrior - Strong melee fighter with high health")
    print("2. 🔥 Mage - Powerful magic user with high intelligence")
    print("3. 🗡 Rogue - Agile, fast, and critical hit specialist")

    while True:
        choice = input("Enter 1, 2, or 3: ").strip()
        name = input("Enter your character name: ").strip()
        
        if choice == "1":
            return Warrior(name)
        elif choice == "2":
            return Mage(name)
        elif choice == "3":
            return Rogue(name)
        else:
            print("⚠ Invalid choice. Please enter 1, 2, or 3.")

# 🎮 Initialize Player & Game Rules
player = choose_player_type()
player.load_profile()  # Load or create player profile
rules = GameRules("Normal", player.role)  # Apply game rules based on player type

# 📦 Create Inventory & Exploration Generator
inventory = Inventory()
exploration = event_generator()

# 🕹 Main Game Loop
def main_menu():
    """Displays the main menu and handles player choices."""
    while True:
        print("\n🔹 Main Menu")
        print("1. Explore the world")
        print("2. View inventory")
        print("3. Enter combat")
        print("4. Save & Exit")
        
        choice = input("Enter your choice: ").strip()

        if choice == "1":
            print(f"🌍 {next(exploration)}")  # Show a random world event
        elif choice == "2":
            print("\n📜 Inventory:")
            if not player.inventory:
                print("❌ Your inventory is empty.")
            else:
                for item in player.inventory:
                    print(f"🔸 {item['name']} - {item['type']}")
        elif choice == "3":
            battle = Combat(player)
            battle.combat_round()
        elif choice == "4":
            player.save_profile()
            print("💾 Progress saved! Exiting game.")
            break
        else:
            print("⚠ Invalid choice. Please try again.")

# 🏁 Start the Game
main_menu()


👜 Steel Sword added to inventory!
A wild beast appears!
A wild beast appears!
You found a treasure chest!
🎮 Game Mode: Hard, Role: Mage
⚔ Enemy Damage Multiplier: 1.7999999999999998
🎮 Game Mode: Hard, Role: Mage
⚔ Enemy Damage Multiplier: 1.7999999999999998

🎭 Choose Your Player Type:
1. 🛡 Warrior - Strong melee fighter with high health
2. 🔥 Mage - Powerful magic user with high intelligence
3. 🗡 Rogue - Agile, fast, and critical hit specialist
📂 Loaded player: Eldrin, Level 1, Role: Mage
🎮 Game Mode: Normal, Role: Mage
⚔ Enemy Damage Multiplier: 1.2

🔹 Main Menu
1. Explore the world
2. View inventory
3. Enter combat
4. Save & Exit

📜 Inventory:
❌ Your inventory is empty.

🔹 Main Menu
1. Explore the world
2. View inventory
3. Enter combat
4. Save & Exit

📜 Inventory:
❌ Your inventory is empty.

🔹 Main Menu
1. Explore the world
2. View inventory
3. Enter combat
4. Save & Exit
🌍 You discovered an ancient ruin.

🔹 Main Menu
1. Explore the world
2. View inventory
3. Enter combat
4. Save & E